# cdo_nco_draft2018a.ipynb

## Purpose
Use CDO and NCO for post-processing of CESM output data, prior to production of figures using [figures_draft2018a.ipynb](https://github.com/grandey/p17d-sulphur-eas-eqm/blob/master/analysis_draft2018a/figures_draft2018a.ipynb) 

## Dependencies
- Climate Data Operators (CDO)
- NetCDF Operators (NCO)
- Python modules mentioned in next cell, including [lib_draft2018a.py](https://github.com/grandey/p17d-sulphur-eas-eqm/blob/master/analysis_draft2018a/lib_draft2018a.py) and its dependencies.

## Data requirements
CESM output data, post-processed to time-series format.

## Author
Benjamin S. Grandey, 2018

In [1]:
! date

import lib_draft2018a
import os

print('lib_draft2018a.py is using {}'.format(lib_draft2018a.dependency_versions()))

Mon Feb 19 17:07:49 +08 2018
lib_draft2018a.py is using {'climapy': '0.1.0+b35937c', 'pandas': '0.22.0', 'xarray': '0.10.0'}


In [2]:
! cdo --version
! ncks --version

Climate Data Operators version 1.9.1 (http://mpimet.mpg.de/cdo)
Compiled: by root on squall2.local (x86_64-apple-darwin17.2.0) Nov  2 2017 18:28:19
CXX Compiler: /usr/bin/clang++ -std=gnu++11 -pipe -Os -stdlib=libc++ -arch x86_64  -D_THREAD_SAFE -pthread
CXX version : unknown
C Compiler: /usr/bin/clang -pipe -Os -arch x86_64  -D_THREAD_SAFE -pthread
C version : unknown
Features: DATA PTHREADS HDF5 NC4/HDF5 OPeNDAP SZ UDUNITS2 PROJ.4 CURL FFTW3 SSE4_1
Libraries: HDF5/1.10.1 proj/4.93 curl/7.56.1
Filetypes: srv ext ieg grb1 nc1 nc2 nc4 nc4c nc5 
     CDI library version : 1.9.1 of Nov  2 2017 18:27:49
 CGRIBEX library version : 1.9.0 of Sep 29 2017 10:16:02
  NetCDF library version : 4.4.1.1 of Oct  6 2017 14:14:42 $
    HDF5 library version : 1.10.1
 SERVICE library version : 1.4.0 of Nov  2 2017 18:27:47
   EXTRA library version : 1.4.0 of Nov  2 2017 18:27:46
     IEG library version : 1.4.0 of Nov  2 2017 18:27:46
    FILE library version : 1.8.3 of Nov  2 2017 18:27:46

NCO netCDF O

In [3]:
scenario_name_dict = lib_draft2018a.load_scenario_name_dict()

In [4]:
in_dir = lib_draft2018a.output_dir  # location of output time-series
out_dir = in_dir  # write files in same directory

## Extract data on specific model levels

In [5]:
variable_ml_forb_list = [('OMEGA', '19', 'b'),  # variable, model level, 'f' or 'b'
                        ('U', '27', 'b')]  # ml 19 is ~525hPa; ml 27 is ~936hPa
for variable, ml, f_or_b in variable_ml_forb_list:
    new_variable = '{}_ml{}'.format(variable, ml)  # new variable name
    for scenario in scenario_name_dict.keys():
        print('{}, {}, {}:'.format(new_variable, f_or_b, scenario))
        # Filenames
        in_filename = '{}/p17d_{}_{}.cam.h0.{}.nc'.format(in_dir, f_or_b,
                                                          scenario, variable)
        out_filename = '{}/p17d_{}_{}.cam.h0.{}.nc'.format(out_dir, f_or_b,
                                                           scenario, new_variable)
        temp_filename = '{}/temp_{}_{}.cam.h0.{}.nc'.format(out_dir, f_or_b,
                                                            scenario, new_variable)
        # Remove previous out file
        if os.path.exists(out_filename):
            print('  Removing {}'.format(out_filename.split('/')[-1]))
            os.remove(out_filename)
        # Interpolate to pressure level using CDO
        print('  Selecting data for model level {}'.format(ml))
        ! cdo -s sellevidx,{ml} {in_filename} {temp_filename}
        # Rename variable using NCO
        print('  Renaming variable to {}'.format(new_variable))
        ! ncrename -v {variable},{new_variable} {temp_filename} {out_filename} >/dev/null 2>/dev/null
        print('  Written {}'.format(out_filename.split('/')[-1]))
        # Remove temporary file
        for filename in [temp_filename, ]:
            print('  Removing {}'.format(filename.split('/')[-1]))
            os.remove(filename)

OMEGA_ml19, b, 2000:
  Removing p17d_b_2000.cam.h0.OMEGA_ml19.nc
  Selecting data for model level 19
  Renaming variable to OMEGA_ml19
  Written p17d_b_2000.cam.h0.OMEGA_ml19.nc
  Removing temp_b_2000.cam.h0.OMEGA_ml19.nc
OMEGA_ml19, b, eas0:
  Removing p17d_b_eas0.cam.h0.OMEGA_ml19.nc
  Selecting data for model level 19
  Renaming variable to OMEGA_ml19
  Written p17d_b_eas0.cam.h0.OMEGA_ml19.nc
  Removing temp_b_eas0.cam.h0.OMEGA_ml19.nc
OMEGA_ml19, b, eas0b:
  Removing p17d_b_eas0b.cam.h0.OMEGA_ml19.nc
  Selecting data for model level 19
  Renaming variable to OMEGA_ml19
  Written p17d_b_eas0b.cam.h0.OMEGA_ml19.nc
  Removing temp_b_eas0b.cam.h0.OMEGA_ml19.nc
OMEGA_ml19, b, eas0c:
  Removing p17d_b_eas0c.cam.h0.OMEGA_ml19.nc
  Selecting data for model level 19
  Renaming variable to OMEGA_ml19
  Written p17d_b_eas0c.cam.h0.OMEGA_ml19.nc
  Removing temp_b_eas0c.cam.h0.OMEGA_ml19.nc
U_ml27, b, 2000:
  Removing p17d_b_2000.cam.h0.U_ml27.nc
  Selecting data for model level 27
  Renaming 

In [6]:
! date

Mon Feb 19 17:09:33 +08 2018
